In [2]:
import pandas as pd

# Replace 'your_file.xlsx' with the path to your Excel file
file_path = '../../storage/Judicial/raw_data_juicios.xlsx'

# Read the Excel file
df = pd.read_excel(file_path)

# transform the cedula column into a string from a float
df['Cédula'] = df['Cédula'].fillna(0).astype(int).astype(str)
df['Cédula'] = df['Cédula'].apply(lambda x: '0' + x if len(x) == 9 else x)

# print head
print(df.head())

       Cédula        id_Juicio                                  Delito  \
0  0100004407  03204201700015               COBRO DE PAGARÉ A LA ORDEN   
1  0100007269  01204201905175                                ALIMENTOS   
2  0100007269  01204201903452                                ALIMENTOS   
3  0100007269  01204201900368   INVESTIGACIÓN DE PATERNIDAD/MATERNIDAD   
4  0100013465  01371202300415                PAGO DE HABERES LABORALES   

         Fecha_de_Ingreso        mes     año                 Estado  \
0 2017-01-09 13:23:24.813    January  2017.0              No activo   
1 2019-09-09 15:50:33.447  September  2019.0  No se pudo determinar   
2 2019-06-12 13:19:18.733       June  2019.0              No activo   
3 2019-01-18 21:50:53.770    January  2019.0              No activo   
4 2023-12-05 17:28:11.537   December  2023.0                 Activo   

  Monto_por_juicio  
0              NaN  
1              NaN  
2              NaN  
3              NaN  
4              NaN  


In [3]:
# get the unique value sof the columns Estado and the count for each
print(df['Estado'].value_counts())

Estado
No activo                522074
Activo                   215943
No se pudo determinar    164947
Name: count, dtype: int64


#### Fist we need to clasify the juicios types into 5 diffrent catgories: Alimentos, Familiares, Penales, Estafas, Cobros

In [4]:
# get all difrent types of value sin the colum 'Delito'
unique_delitos = df['Delito'].unique()
# make sure that all of them are strings
unique_delitos = [str(d) for d in unique_delitos]

print(unique_delitos)
print(len(unique_delitos))

['COBRO DE PAGARÉ A LA ORDEN', 'ALIMENTOS', 'INVESTIGACIÓN DE PATERNIDAD/MATERNIDAD', 'PAGO DE HABERES LABORALES', 'PRESCRIPCIÓN ADQUISITIVA DE DOMINIO', 'PRESCRIPCIÓN EXTRAORDINARIA ADQUISITIVA DE DOMINIO', 'LESIONES', 'AMPARO POSESORIO', 'DERECHOS FUNDAMENTALES DEL CONSUMIDOR', 'ARCHIVO DE LA INVESTIGACIÓN PREVIA', 'OPOSICIÓN A LA RENDICIÓN DE CUENTAS', 'COBRO DE DINERO', 'REINVINDICACIÓN', 'RESOLUCIÓN DE CONTRATO', 'COBRO DE FACTURAS POR BIENES Y SERVICIOS, HONORARIOS PROFESIONALES CUANDO LA PRETENSIÓN NO SEA EXIGIBLE EN PROCEDIMIENTO MONITORIO O EN VIA EJECUTIVA', 'TERMINACIÓN DE CONTRATO', 'COBRO DE LETRA DE CAMBIO', 'CONTRAVENCIONES DE TRÁNSITO', 'CONTRATO DE MUTUO O PRESTAMO', 'DIVORCIO POR CAUSAL', 'INCUMPLIMIENTO DE DECISIONES LEGÍTIMAS DE AUTORIDAD COMPETENTE', 'COBRO DE HONORARIOS DE ABOGADO', 'PAGO POR CONSIGNACIÓN', 'FACTURAS O DOCUMENTOS ART. 356 NUM.2', 'FACTURA O PRIMA DE SEGUROS', 'PARTICION NO VOLUNTARIA', 'APLICACIÓN DEL PRINCIPIO DE OPORTUNIDAD', 'NULIDAD DEL ACTA D

### Classify each type of delito 

In [5]:
# filter 
def filter_delitos(list1, list2):
    matched = []
    remaining = []
    for element in list1:
        if element in list2:
            matched.append(element)
        else:
            remaining.append(element)
    return matched, remaining

In [6]:
import json

with open("classified_delitos.json", 'r', encoding='utf-8') as file:
    categories = json.load(file)

print(categories)

categories_delitos = {}
remaing_delitos = unique_delitos
for key, value in categories.items():
    categories_delitos[key], remaing_delitos = filter_delitos(remaing_delitos, value)
    print(key, len(categories_delitos[key]) )

print('remaing_delitos', remaing_delitos)
print('remaing_delitos', len(remaing_delitos))

{'penal': ['LESIONES', 'CONTRAVENCIONES DE TRÁNSITO', 'INCUMPLIMIENTO DE DECISIONES LEGÍTIMAS DE AUTORIDAD COMPETENTE', 'CONTRAVENCIONES DE CUARTA CLASE', 'MUERTE CULPOSA', 'DEFRAUDACIÓN TRIBUTARIA', 'COHECHO', 'PECULADO', 'DESPOJO VIOLENTO', 'ABUSO DE CONFIANZA', 'FALSIFICACIÓN Y USO DE DOCUMENTO FALSO', 'CALUMNIA', 'CONDUCCIÓN DE VEHÍCULO EN ESTADO DE EMBRIAGUEZ', 'PERJURIO Y FALSO TESTIMONIO', 'TRÁFICO ILÍCITO DE SUSTANCIAS CATALOGADAS SUJETAS A FISCALIZACIÓN', 'RECEPTACIÓN', 'MUERTE A ANIMAL QUE FORMA PARTE DEL ÁMBITO DE LA FAUNA URBANA', 'SECUESTRO', 'TENENCIA Y PORTE DE ARMAS', 'INTIMIDACIÓN', 'DROGAS', 'ROBO', '297 ENRIQUECIMIENTO PRIVADO NO JUSTIFICADO', 'CONTRAVENCIÓN DE HURTO', '369 DELICUENCIA ORGANIZADA INC 2', 'CAPTACIÓN ILEGAL DE DINERO', 'CONTRAVENCIONES DE PRIMERA CLASE', 'ATAQUE O RESISTENCIA', 'VIOLACIÓN DE PROPIEDAD PRIVADA', 'DELINCUENCIA ORGANIZADA', '361 ARMAS DE FUEGO, MUNICIONES Y EXPLOSIVOS NO AUTORIZADOS', 'APREHENSIÓN DEL AUTOMOTOR POR ASUNTO CIRCULAR POR EL 

### Let's organize the df by cedula

In [7]:
# drop cedulas with null values or 0
df = df[df['Fecha_de_Ingreso'] != '0']
df_sorted = df.sort_values('Fecha_de_Ingreso')
print(df_sorted.head())

            Cédula        id_Juicio                            Delito  \
304478  0920960317  09284201604877                   DAÑOS MATERIALES   
57040   0400929089  02281201600514                          SECUESTRO   
885633  2100132956  02281201600514                          SECUESTRO   
349808  0940236854  09287201602095    CONTRAVENCIONES DE CUARTA CLASE   
793787  1750647693  23303201700002   CONTRAVENCIONES DE PRIMERA CLASE   

              Fecha_de_Ingreso      mes     año     Estado Monto_por_juicio  
304478 2017-01-01 00:16:49.480  January  2017.0  No activo              400  
57040  2017-01-01 01:00:12.560  January  2017.0     Activo             9000  
885633 2017-01-01 01:00:12.560  January  2017.0  No activo              NaN  
349808 2017-01-01 01:33:40.803  January  2017.0  No activo          $100,00  
793787 2017-01-01 05:00:00.000  January  2017.0  No activo              NaN  


#### We loop thourhg each row sorted by the date, we classify the delitos for each cedula and keep track of the count of the delitos classfied

In [9]:
# Sort the dataframe by the 'Fecha_de_Ingreso' column
df_sorted = df.sort_values('Fecha_de_Ingreso')
columns = [ 
    'penal-activos', 'penal-inactivos', 'penal-N/A'
    'alimentos-activos', 'alimentos-inactivos', 'alimentos-N/A',
    'familiares-activos', 'familiares-inactivos', 'familiares-N/A',
    'estafa-activos', 'estafa-inactivos', 'estafa-N/A',
    'robo-activos', 'robo-inactivos', 'robo-N/A',
    'cobros-activos', 'cobros-inactivos', 'cobros-N/A',
    'otros-activos', 'otros-inactivos', 'otros-N/A',
    'total-activos', 'total-inactivos', 'total-N/A'
]
# make a new df to add the rows with the classified delitos
df_aggregate = []

# Create a dictionary to store the count of classified delitos for each cedula
classified_delitos_count = {}

# Loop through each row in the sorted dataframe
for index, row in df_sorted.iterrows():
    fecha = row['Fecha_de_Ingreso']
    cedula = row['Cédula']
    delito = row['Delito']
    estado = row['Estado']
    
    # Classify the delito and update the count in the dictionary
    for category, list in categories_delitos.items():
        if delito in categories_delitos[category]:
            # choose a category
            category = category
            # add estado
            if(estado == 'Activo'):
                category = category + '-activos'
            elif(estado == 'No activo'):
                category = category + '-inactivos'
            else:
                category = category + '-N/A'
            break

    if cedula not in classified_delitos_count:
        classified_delitos_count[cedula] = {
            'penal-activos': 0, 'penal-inactivos': 0, 'penal-N/A': 0,
            'alimentos-activos': 0, 'alimentos-inactivos': 0, 'alimentos-N/A': 0,
            'familiares-activos': 0, 'familiares-inactivos': 0, 'familiares-N/A': 0,
            'estafa-activos': 0, 'estafa-inactivos': 0, 'estafa-N/A': 0,
            'robo-activos': 0, 'robo-inactivos': 0, 'robo-N/A': 0,
            'cobros-activos': 0, 'cobros-inactivos': 0, 'cobros-N/A': 0,
            'otros-activos': 0, 'otros-inactivos': 0, 'otros-N/A': 0,
            'total-activos': 0, 'total-inactivos': 0, 'total-N/A': 0
        }
    else:
        if category not in classified_delitos_count[cedula]:
            classified_delitos_count[cedula][category] = 0
        classified_delitos_count[cedula][category] += 1
        if estado == 'Activos':
            classified_delitos_count[cedula]['total-activos'] += 1
        elif estado == 'Inactivos':
            classified_delitos_count[cedula]['total-inactivos'] += 1
        else:
            classified_delitos_count[cedula]['total-N/A'] += 1

    data = { 
        'Fecha_de_Ingreso': fecha,
        'Cédula': cedula,
        **classified_delitos_count[cedula],
    }
    # add the row to the df_aggregate
    df_aggregate.append(pd.DataFrame([data]))

pd.concat(df_aggregate).to_csv('judicial_aggregate.csv', index=False)

